# Texas Comptroller of Public Accounts - Scraper

## Importing Libraries

In [1]:
# Importing Libraries
from selenium import webdriver
from selenium.webdriver.remote.webdriver import WebDriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup

from io import StringIO

import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


import numpy as np

from typing import Optional

import time

## Creating Scraper Class

In [2]:
class _LeaseDropNaturalGas_WebScraper:

    def __init__(self) -> None:
        self.site_key: str = '6Lf6Z5sUAAAAACg7ECAeRMcnAo2_WfoKUeNYXkj_'
        self.login_url: str = 'https://mycpa.cpa.state.tx.us/cong/loginForward.do?phase=check'
        self.ngl_drop_url: str = 'https://mycpa.cpa.state.tx.us/cong/leaseDropNGAction.do'
        self.xpath_leaseNo: str = '//*[@id="leaseNum"]'
        self.xpath_begDt: str = '//*[@id="begFilPrd"]'
        self.xpath_endDt: str = '//*[@id="endFilPrd"]'
        self.xpath_submitForm: str = '//*[@id="leaseDropNGForm"]/span[7]/p/input'
        self.xpath_lease_table: str = '//*[@id="menucontenttable"]/table/tbody/tr/td[2]/div/table'
        self.driver: WebDriver  = None
        self._initialize_driver()


    def _initialize_driver(self) -> None:
        """
        Initializes the Chrome WebDriver.
        """
        options = webdriver.ChromeOptions()
        # options.add_argument('--headless')  # Optional: run in headless mode
        options.add_argument('--disable-gpu')  # Optional: disable GPU
        options.add_argument('--no-sandbox')  # Optional: required for some environments

        self.driver = webdriver.Chrome(options=options)


    def _load_page(self) -> None:
        """
        Load the login and Natural Gas Inquiry drop page.
        """
        if self.driver is None:
            raise RuntimeError("WebDriver is not initialized.")
        
        self.driver.maximize_window()
        self.driver.get(self.login_url)
        time.sleep(0.8)
        self.driver.get(self.ngl_drop_url)
        wait = WebDriverWait(self.driver, 1)
        wait.until(lambda d: d.execute_script("return typeof grecaptcha !== 'undefined'"))


    def _get_recaptcha_token(self) -> str:
        """
        Retrieve the reCAPTCHA token from the webpage.

        Returns:
            Optional[str]: The reCAPTCHA token as a string, or None if the token retrieval fails.
        """
        
        if self.driver is None:
            raise RuntimeError("WebDriver is not initialized.")

        self._load_page()
        
        token = self.driver.execute_script(f'''
            return grecaptcha.execute('{self.site_key}', {{action: 'homepage'}}).then(function(token) {{
                return token;
            }});
        ''')
        
        return token


    def _get_NGL_Inquiry_html(self, lease_no: str, beg_dt: str, end_dt: str) -> str:
        """
        Scrape the Natural Gas Inquiry form based on lease_no, beg_dt, and end_dt.
        
        Args:
            lease_no (str): The lease number to search (6 or all digits).
            beg_dt (str): Begining period (yymm or yy)
            end_dt (str): Ending period (yymm or yy)
        
        Returns:
            Optional[str]: The HTML content of the page or None if an error occurred.
        """

        if self.driver is None:
            raise RuntimeError("WebDriver is not initialized.")
        
        # Format Lease Number
        try:
            if len(lease_no) == 11:
                lease_no = lease_no.split('-')[1]
            
            elif len(lease_no) == 6:
                lease_no = lease_no

        except ValueError:
            print(f'Lease number entered not of 6 or 11 digits')
        
        # Loading the page
        self._load_page()

        # Filling Lease Number
        self.driver.find_element(By.XPATH, self.xpath_leaseNo).send_keys(lease_no)

        # Filling Begining Period
        self.driver.find_element(By.XPATH, self.xpath_begDt).send_keys(beg_dt)

        # Filling Ending Period
        self.driver.find_element(By.XPATH, self.xpath_endDt).send_keys(end_dt)

        # Running the Inquiry Form
        time.sleep(0.5)
        self.driver.find_element(By.XPATH, self.xpath_submitForm).click()

        # Visibility of the table header element
        try:
            # Wait until the table is located or timeout occurs
            lease_table = WebDriverWait(self.driver, 10).until(
                EC.presence_of_all_elements_located(
                    (By.XPATH, self.xpath_lease_table)
                )
            )

            if lease_table:
                return self.driver.page_source
            
        except Exception as e:
            print("Lease table not found.",e)
            return False
    

    def _clear_entry_labels(self) -> None:
        """
        Clear the input fields for Lease Number, Beginning Period, and Ending Period.

        Returns:
            None: This function does not return anything.
        """
        # Clearing Lease Number
        self.driver.find_element(By.XPATH, self.xpath_leaseNo).clear()

        # Clearing Begining Period
        self.driver.find_element(By.XPATH, self.xpath_begDt).clear()

        # Clearing Ending Period
        self.driver.find_element(By.XPATH, self.xpath_endDt).clear()


    def _parse_html(self, html: str, df_raw: bool = False) -> pd.DataFrame:
        """
        Parsing HTML content using Beautiful Soup into a DataFrame
        
        Args:
            html (str): The HTML content as string.
        
        Returns:
            pd.DataFrame: The parsed and cleaned DataFrame.
        """
        soup = BeautifulSoup(html,'html.parser')

        df_raw = pd.read_html(StringIO(str(soup.find_all('table'))))

        df_LeaseNGL_raw = df_raw[1]

        # Step 1: Identify rows where 'Primary Taxpayer #' contains 'Period' and extract the date part
        df_LeaseNGL_raw['prod_dt'] = np.where(
            df_LeaseNGL_raw['Primary Taxpayer #'].str.contains('Period', na=False),
            df_LeaseNGL_raw['Primary Taxpayer #'].str.extract(r'Period: (\d{4})', expand=False),
            np.nan
        )

        # Step 2: Forward fill the 'prod_dt' column to propagate the last valid date value
        df_LeaseNGL_raw['prod_dt'] = df_LeaseNGL_raw['prod_dt'].ffill()


        # Step 3: Convert 'prod_dt' from 'YYMM' to datetime format 'YYYY-MM-DD'
        df_LeaseNGL_raw['prod_dt'] = pd.to_datetime(df_LeaseNGL_raw['prod_dt'], format='%y%m')

        # Step 4: Filter out rows where column 'Primary Taxpayer #' contains 'Period'
        df_LeaseNGL_cleaned = df_LeaseNGL_raw[~df_LeaseNGL_raw['Primary Taxpayer #'].str.contains('Period', na=False)].reset_index(drop=True)

        # Step 5: Clean column names
        df_LeaseNGL_cleaned.columns = df_LeaseNGL_cleaned.columns.str.lower()  # Convert to lowercase
        df_LeaseNGL_cleaned.columns = df_LeaseNGL_cleaned.columns.str.replace('#', '')  # Remove '#' character
        df_LeaseNGL_cleaned.columns = df_LeaseNGL_cleaned.columns.str.replace(' ', '_')  # Replace spaces with underscores

        return df_LeaseNGL_cleaned


    def _quit(self) -> None:
        """
        Close and quit the WebDriver.
        """
        if self.driver is not None:
            self.driver.close()
            self.driver.quit()
            self.driver = None

In [5]:
# Testing the scraper

scraper = _LeaseDropNaturalGas_WebScraper()

try:
    # Fill the form and get the HTML content
    html_content = scraper._get_NGL_Inquiry_html(lease_no='017147', beg_dt='2301', end_dt='2301')

    # Parse the HTML and get the cleaned DataFrame
    if html_content:
        df = scraper._parse_html(html=html_content)
finally:
    scraper._quit()

In [6]:
df

,sub_type,primary_taxpayer_,comm_code,lse_typ,cnty/_dpi,exmt_typ,api_nbr,off_lease,other_party_taxpayer,secondary_tp_name,tax_reimb,ttl_lease_volume,your_volume,your_value,tax_due,gr_volume,gr_value,marketing_cost,net_tax_value,tax_rate,05_tax_due,error_status,prod_dt
0,Pro,13646174337,RG,OIL,7/NO,NaN,NaN,YES,NaN,NaN,NO,48.0,48.0,$203.89,YES,0.0,$0.00,$359.68,$0.00,0.0,$0.00,NO,2023-01-01
1,Pro,17523477697,RG,OIL,165/NO,NaN,NaN,NO,1.752348e+10,XTO ENERGY INC.,NO,0.0,1755.0,"$12,838.14",YES,2.0,$14.46,"$12,823.68",$0.00,0.0,$0.00,NO,2023-01-01
2,Pro,17523477697,RS,OIL,165/NO,NaN,NaN,NO,1.752348e+10,XTO ENERGY INC.,NO,0.0,34603.0,"$196,970.27",YES,39.0,$222.02,"$28,148.04","$168,600.21",0.0,$0.00,NO,2023-01-01
3,Pro,17523477697,PR,OIL,165/NO,NaN,NaN,NO,1.135402e+10,EXXONMOBIL OIL CORPORATION,NO,0.0,41720.0,"$106,352.34",YES,47.0,$120.05,"$12,843.66","$93,388.63",0.0,$0.00,NO,2023-01-01
4,Pro,17523477697,PR,OIL,165/NO,NaN,NaN,NO,1.201088e+10,"ENERGY TRANSFER FUEL, LP",NO,0.0,544.0,"$12,356.51",YES,1.0,$13.98,"$1,443.20","$10,899.33",0.0,$0.00,NO,2023-01-01
5,Pro,17523477697,RG,OIL,231/NO,NaN,NaN,NO,1.752348e+10,XTO ENERGY INC.,NO,0.0,97433.0,"$503,335.31",YES,110.0,$565.31,"$107,957.68","$394,812.32",0.0,$0.00,NO,2023-01-01
6,Pro,17523477697,RG,OIL,231/NO,NaN,NaN,NO,1.760508e+10,TARGA MIDSTREAM SERVICES LLC,YES,0.0,57271.0,"$410,150.22",YES,65.0,$467.48,"$28,552.86","$381,129.88",0.0,$0.00,NO,2023-01-01
7,Pro,17523477697,PR,OIL,231/NO,NaN,NaN,NO,1.135402e+10,EXXONMOBIL OIL CORPORATION,NO,0.0,19867.0,"$298,299.83",YES,23.0,$336.69,"$3,428.65","$294,534.49",0.0,$0.00,NO,2023-01-01
8,Pro,17523477697,PR,OIL,231/NO,NaN,NaN,NO,1.201088e+10,"ENERGY TRANSFER FUEL, LP",NO,0.0,1789.0,"$56,038.52",YES,3.0,$63.35,$309.05,"$55,666.12",0.0,$0.00,NO,2023-01-01


## Reading Well Header Data from CC

In [6]:
# Reading well header csv to pandas DataFrame
df_wellheader_raw = pd.read_csv('well_header.csv',low_memory=False)

# Cleaning up the column names
df_wellheader_modified = df_wellheader_raw.copy() # Creating
df_wellheader_modified.columns = df_wellheader_modified.columns.str.lower().str.replace(' ', '_')  # Convert to lowercase and replace spaces with underscores

In [7]:
# Check the length of the values in lease_number columns
# df_wellheader_modified['lease_number'].apply(lambda x: len(str(x)) if pd.notnull(x) else 0).unique()

In [15]:
df_wellheader_modified['lease_number'].unique()

array(['7C-017147-O', '08-P15667-O', '08-056136-O', ..., '08-060433-O',
       '08-060370-O', '7C-022536-O'], dtype=object)

In [8]:
df_wellheader_modified.groupby(['lease_number'])['first_prod_date'].min().to_frame().reset_index()

array([11,  5])